In [1]:
import pandas as pd
import time
import requests
import json
from tqdm import tqdm

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--start-maximized")
options.add_argument("headless")
s = Service(ChromeDriverManager().install())
chrome = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/106.0.5249.61/chromedriver] found in cache


In [2]:
chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=1&cat=%A5b%BE%C9%C5%E9&form=menu&form_id=stock_id&form_name=stock_name&domain=0")

In [3]:
ticker = {}
td_text = chrome.find_elements(by = By.TAG_NAME, value = "td")[3].text
tiker_row = td_text.split("\n")
tiker_row[0] = tiker_row[0].replace("上市 ", "")
tiker_row[len(tiker_row) - 1] = tiker_row[len(tiker_row) - 1][:-18]

In [4]:
for row in tiker_row:
    cols = row.split(" ")

    for col in cols: ticker[col] = []

In [5]:
import json
# list(ticker.keys())

In [6]:
for key in tqdm(ticker.keys()):
    chrome.get("https://tw.stock.yahoo.com/h/kimosel.php?tse=1&cat=" + key + "&form=menu&form_id=stock_id&form_name=stock_name&domain=0")
    tables_tag = chrome.find_elements(by = By.TAG_NAME, value = "table")[6]
    a_tags = tables_tag.find_elements(by = By.TAG_NAME, value = "a")

    for a_tag in a_tags: ticker[key].append(a_tag.text)

    time.sleep(0.5)


100%|██████████| 40/40 [13:06<00:00, 19.65s/it] 


In [7]:
a = []
b = []
c = []

for key in ticker.keys():
    for ele in ticker[key]:
        a.append(ele)
        b.append(ele.split(" ")[0])

    c.extend([key for i in range(len(ticker[key]))])

In [8]:
df = pd.DataFrame({ "stock_name" : a, "stock_num" : b, "class" : c })
df

,stock_name,stock_num,class
0,1101 台泥,1101,水泥
1,1101B 台泥乙特,1101B,水泥
2,1102 亞泥,1102,水泥
3,1103 嘉泥,1103,水泥
4,1104 環泥,1104,水泥
...,...,...,...
25875,6912 益鈞環科*-新,6912,戰略新板
25876,6913 鴻呈-新,6913,戰略新板
25877,6922 宸曜-新,6922,戰略新板
25878,6925 意藍-新,6925,戰略新板


In [16]:
df.to_csv("ticker_list.csv", index = False)